In [1]:
import pandas as pd 
from scipy import stats

In [2]:
df = pd.read_csv('AB_test_results.csv')

In [3]:
df.head()

,user_id,install_date,country,test_group,trial,paid,subscription_name,revenue_1m
0,0000dd3fa4702a63d1b76aaffe1ab39b,2023-06-05,US,treatment,0,0,NaN,0.0
1,0001f27ab7e22228e54c8b2028b43f24,2023-06-07,AU,treatment,0,0,NaN,0.0
2,0006c5c547801308b36ea3cf669856ae,2023-06-07,AU,treatment,0,0,NaN,0.0
3,000d1a300263c5db91cbefa3852898a7,2023-06-07,AU,control,0,0,NaN,0.0
4,000e5e62e8746e467ed9f49ac5de3208,2023-06-06,US,treatment,0,0,NaN,0.0


In [4]:
df.shape

(15285, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15285 entries, 0 to 15284
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            15285 non-null  object 
 1   install_date       15285 non-null  object 
 2   country            15285 non-null  object 
 3   test_group         15285 non-null  object 
 4   trial              15285 non-null  int64  
 5   paid               15285 non-null  int64  
 6   subscription_name  1723 non-null   object 
 7   revenue_1m         15285 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 955.4+ KB


In [6]:
# Number of records for each country
df.groupby('country').count()

,user_id,install_date,test_group,trial,paid,subscription_name,revenue_1m
country,,,,,,,
AU,3763,3763,3763,3763,3763,419,3763
CA,3895,3895,3895,3895,3895,427,3895
GB,3837,3837,3837,3837,3837,479,3837
US,3790,3790,3790,3790,3790,398,3790


In [7]:
# Number of users in each group
df[['test_group', 'user_id', 'subscription_name']].groupby('test_group').count()

,user_id,subscription_name
test_group,,
control,7627,925
treatment,7658,798


In [8]:
# Number of users in each group
control_users = df[df['test_group'] == 'control'][['user_id']].count()
print(control_users)
treatment_users = df[df['test_group'] == 'treatment'][['user_id']].count()
print(treatment_users)

user_id    7627
dtype: int64
user_id    7658
dtype: int64


In [9]:
# Number of trial subscriptions in each group
treatment_users_subs = df[df['test_group'] == 'treatment'][['subscription_name']].count()
control_users_subs = df[df['test_group'] == 'control'][['subscription_name']].count()
print(treatment_users_subs)
print(control_users_subs)

subscription_name    798
dtype: int64
subscription_name    925
dtype: int64


In [10]:
treatment_users = int(treatment_users)
treatment_users_subs = int(treatment_users_subs)
control_users = int(control_users)
control_users_subs = int(control_users_subs)

C:\Users\ironf\AppData\Local\Temp\ipykernel_13520\1790847438.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  treatment_users = int(treatment_users)
C:\Users\ironf\AppData\Local\Temp\ipykernel_13520\1790847438.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  treatment_users_subs = int(treatment_users_subs)
C:\Users\ironf\AppData\Local\Temp\ipykernel_13520\1790847438.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  control_users = int(control_users)
C:\Users\ironf\AppData\Local\Temp\ipykernel_13520\1790847438.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  control_users_subs = int(control_users_subs)


In [11]:
# Percentage of trial subscriptions in each group 
treatment_subs_perc = round(treatment_users_subs/treatment_users * 100, 2)
cuntrol_subs_perc = round(control_users_subs/control_users * 100, 2)
print('treatment: ' , treatment_subs_perc)
print('control: ' , cuntrol_subs_perc)

treatment:  10.42
control:  12.13


#### The percentage of new trial subscriptions is 1.7% lower

In [12]:
# Number of subscriptions and amount of rhubarb in each country for the control group of users
df[df['test_group'] == 'control'][['country', 'subscription_name', 'revenue_1m']].groupby('country').agg({'subscription_name': ['count'], 'revenue_1m':['sum']})

,subscription_name,revenue_1m
,count,sum
country,,
AU,228,179.70
CA,219,107.82
GB,270,209.65
US,208,197.67


In [13]:
# The number of subscriptions and the amount of revenue in each country for the treatment group of users
df[df['test_group'] == 'treatment'][['country', 'subscription_name', 'revenue_1m']].groupby('country').agg({'subscription_name': ['count'], 'revenue_1m':['sum']})

,subscription_name,revenue_1m
,count,sum
country,,
AU,191,209.30
CA,208,185.38
GB,209,221.26
US,190,176.41


In [14]:
# Number of full subscriptions for the control group
df[df['test_group'] == 'control']['paid'].sum()

116

In [15]:
# Number of full subscriptions for the treatment group
df[df['test_group'] == 'treatment']['paid'].sum()

149

In [16]:
# Comparison of statistics for each group 
df[['test_group', 'trial', 'paid', 'subscription_name', 'revenue_1m']].groupby('test_group').agg({'trial':['sum'],'paid':['sum'], 'subscription_name': ['count'], 'revenue_1m':['sum']})

,trial,paid,subscription_name,revenue_1m
,sum,sum,count,sum
test_group,,,,
control,925,116,925,694.84
treatment,798,149,798,792.35


In [17]:
# Comparison of statistics for each group and country 
summary = df[['test_group', 'country', 'trial', 'paid', 'subscription_name', 'revenue_1m']].groupby(['test_group', 'country']).agg({'trial':['sum'],'paid':['sum'], 'subscription_name': ['count'], 'revenue_1m':['sum']})
summary

trial paid subscription_name revenue_1m
                     sum  sum             count        sum
test_group country                                        
control    AU        228   30               228     179.70
           CA        219   18               219     107.82
           GB        270   35               270     209.65
           US        208   33               208     197.67
treatment  AU        191   39               191     209.30
           CA        208   37               208     185.38
           GB        209   41               209     221.26
           US        190   32               190     176.41

In [18]:
summary[('conversion_rate', 'paid_per_trial')] = summary[('paid', 'sum')] / summary[('trial', 'sum')]
summary

trial paid subscription_name revenue_1m conversion_rate
                     sum  sum             count        sum  paid_per_trial
test_group country                                                        
control    AU        228   30               228     179.70        0.131579
           CA        219   18               219     107.82        0.082192
           GB        270   35               270     209.65        0.129630
           US        208   33               208     197.67        0.158654
treatment  AU        191   39               191     209.30        0.204188
           CA        208   37               208     185.38        0.177885
           GB        209   41               209     221.26        0.196172
           US        190   32               190     176.41        0.168421

**There is a strong difference in the number of full subscriptions in Canada. 18 (8.2%) in the control group and 37 (17.7%) in the group with new subscriptions**

In [19]:
treatment_users_full_subs = df[(df['test_group'] == 'treatment') & (df['paid'] == 1)][['user_id']].count()
control_users_full_subs = df[(df['test_group'] == 'control') & (df['paid'] == 1)][['user_id']].count()

treatment_users_full_subs = int(treatment_users_full_subs)
control_users_full_subs = int(control_users_full_subs)

print(treatment_users_full_subs)
print(control_users_full_subs)

149
116


C:\Users\ironf\AppData\Local\Temp\ipykernel_13520\595580691.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  treatment_users_full_subs = int(treatment_users_full_subs)
C:\Users\ironf\AppData\Local\Temp\ipykernel_13520\595580691.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  control_users_full_subs = int(control_users_full_subs)


In [20]:
# Percentage of full subscriptions (vs. trial subscriptions) among different groups
treatment_users_full_subs_perc = treatment_users_full_subs/treatment_users_subs * 100
control_users_full_subs_perc = control_users_full_subs/control_users_subs * 100
print('treatment % full sub: ', treatment_users_full_subs_perc)
print('control % full sub: ', control_users_full_subs_perc)

treatment % full sub:  18.671679197994987
control % full sub:  12.54054054054054


#### The renewal rate of new subscriptions is about 1.5 times higher

### Significance testing (Hypothesis testing)

$H_0$: The average value of full subscription payment in the control group and the treatment group are equal.

$H_A$: The average value of full subscription payment in one of the groups differs from the average value in the other group

**Significance level:**
<br>I chose a **5%** significance level for this project

In [53]:
control_paid = df[df['test_group'] == 'control'].paid

In [56]:
treatment_paid = df[df['test_group'] == 'treatment'].paid

In [55]:
stats.ttest_ind(a = control_paid, b = treatment_paid, equal_var = False)

TtestResult(statistic=-2.0123031985781297, pvalue=0.04420573349823753, df=15078.789085480468)

Since the p-value is less than the chosen significance level of 5%, I reject the null hypothesis. This indicates that there is a statistically significant difference between the control and treatment groups.